In [2]:
# Libraries imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
import pickle

In [3]:
#test data
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv').set_index('Id')
## removing missing values
test = test.dropna()
## converting categorical variables
test = test.replace({'EJ' : { 'A' : 0, 'B' : 1}})
## Scaling
test =pd.DataFrame(MinMaxScaler().fit_transform(test), columns=test.columns)
## Predict_Prob
SGDClassifier_model = pickle.load(open('/kaggle/input/model-l1/SGDClassifier_model_l1.pkl', 'rb'))
## Creating Submission_file
submission = pd.DataFrame(SGDClassifier_model.predict_proba(test), 
                          columns=['class_0','class_1'])

submission.insert(0,
                  'Id',
                  pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')['Id'])

submission.to_csv('submission.csv', index=False)